# Preparação do Dataset

## Montar o Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Checar se T4 GPU está ativa

In [1]:
!nvidia-smi

Mon Apr  7 17:28:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Aplicar Augumentação

In [ ]:
!pip install albumentations opencv-python-headless

In [ ]:
import os
import cv2
from pathlib import Path
from albumentations import Compose, HorizontalFlip, RandomBrightnessContrast, MotionBlur, Affine, CoarseDropout, BboxParams
from collections import defaultdict

# Classes minoritárias alvo (índices)
CLASSES_ALVO = [1, 2, 4, 5, 6, 7]
MAX_IMGS_POR_CATEGORIA = 1000
N_AUGS = 3

# Contador de imagens por categoria
contador_imgs = defaultdict(int)

# Caminhos
IMGS_DIR = Path("content/drive/MyDrive/dataset/train/images")
LABELS_DIR = Path("content/drive/MyDrive/dataset/train/labels")

# Augmentações definidas
transform = Compose([
    HorizontalFlip(p=0.5),
    RandomBrightnessContrast(p=0.5),
    MotionBlur(blur_limit=3, p=0.2),
    Affine(translate_percent=0.05, scale=1.1, rotate=15, p=0.5),
    CoarseDropout(
        num_holes_range=(1, 4),
        hole_height_range=(0.05, 0.2),
        hole_width_range=(0.05, 0.2),
        fill=0,
        p=0.3)
],
bbox_params=BboxParams(
    format='yolo',
    label_fields=['class_labels'],
    min_visibility=0.1,
    clip=True,
    filter_invalid_bboxes=True
))

def carrega_labels(label_path):
    with open(label_path, 'r') as f:
        linhas = f.readlines()
    boxes = []
    class_ids = []
    for linha in linhas:
        linha = linha.strip()
        if not linha:
            continue
        partes = linha.split()
        if len(partes) < 5:
            continue
        try:
            cid = int(partes[0])
            bbox = list(map(float, partes[1:5]))
            boxes.append(bbox)
            class_ids.append(cid)
        except ValueError:
            continue
    return boxes, class_ids

def salvar_labels(bboxes, class_ids, out_path):
    with open(out_path, 'w') as f:
        for cls, box in zip(class_ids, bboxes):
            if box[2] <= 0 or box[3] <= 0:
                continue
            f.write(f"{int(cls)} {' '.join(f'{x:.6f}' for x in box)}\n")

# Processar imagens
for img_file in IMGS_DIR.glob("*.jpg"):
    label_file = LABELS_DIR / img_file.with_suffix('.txt').name
    if not label_file.exists():
        continue

    bboxes, class_ids = carrega_labels(label_file)

    # Só considera se houver classe de interesse
    if not any(cls in CLASSES_ALVO for cls in class_ids):
        continue

    # Pula se todas as classes da imagem já bateram o limite
    if all(contador_imgs[cls] >= MAX_IMGS_POR_CATEGORIA for cls in class_ids if cls in CLASSES_ALVO):
        continue

    image = cv2.imread(str(img_file))
    h, w = image.shape[:2]

    for i in range(N_AUGS):
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_ids)
        aug_img = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_classes = augmented['class_labels']

        if len(aug_bboxes) == 0:
            continue

        # Verifica se pode salvar (pelo menos uma classe da imagem está abaixo do limite)
        if all(contador_imgs[cls] >= MAX_IMGS_POR_CATEGORIA for cls in aug_classes if cls in CLASSES_ALVO):
            continue

        new_img_name = img_file.stem + f"_aug{i}.jpg"
        new_lbl_name = label_file.stem + f"_aug{i}.txt"

        cv2.imwrite(str(IMGS_DIR / new_img_name), aug_img)
        salvar_labels(aug_bboxes, aug_classes, LABELS_DIR / new_lbl_name)

        # Atualiza contadores apenas das classes realmente salvas
        for cls in set(aug_classes):
            if cls in CLASSES_ALVO:
                contador_imgs[cls] += 1

print("✅ Superaugmentações concluídas com sucesso.")


## Visualizar a distribuição do dataset

In [ ]:
import os
from glob import glob
from collections import defaultdict

# 📁 Caminho base onde estão os diretórios train/valid/test
dataset_base = "content/drive/MyDrive/dataset"

class_names = ['knife', 'scissor', 'scalpel', 'axe', 'saw', 'chainsaw', 'chisel', 'sickle']

# 🔁 Inicializa contadores por split
image_counts_by_split = {
    "train": defaultdict(set),
    "valid": defaultdict(set),
    "test": defaultdict(set)
}

# 🔍 Função para processar cada split
def count_images_per_category(split_name):
    label_dir = os.path.join(dataset_base, split_name, "labels")
    for label_file in glob(f"{label_dir}/**/*.txt", recursive=True):
        with open(label_file, 'r') as f:
            content = f.read()
            categories_in_image = set()
            for line in content.strip().splitlines():
                parts = line.strip().split()
                if parts and parts[0].isdigit():
                    class_id = int(parts[0])
                    if 0 <= class_id < len(class_names):
                        categories_in_image.add(class_names[class_id])
            for category in categories_in_image:
                image_counts_by_split[split_name][category].add(label_file)

# 🧭 Percorre os splits
for split_key in ['train', 'valid', 'test']:
    print(f"🔍 Contando imagens em: {split_key}")
    count_images_per_category(split_key)

# 📊 Exibe resultados como tabela Markdown por split
print("\n### 📸 Quantidade de imagens por categoria e por split\n")

for split_key in ['train', 'valid', 'test']:
    print(f"\n#### 📂 {split_key.capitalize()}")
    print("| Categoria    | Imagens únicas |")
    print("|--------------|----------------|")
    for class_name in class_names:
        count = len(image_counts_by_split[split_key][class_name])
        print(f"| {class_name:<12} | {count:<14} |")


# Treinamento do Modelo

In [ ]:
!pip install ultralytics

In [ ]:
# 📌 Importar
from ultralytics import YOLO

# 📌 Carregar modelo YOLOv8s
model = YOLO('yolov8s.yaml')  # ou 'yolov8s.pt' para transfer learning

# 📌 Treinar o modelo
model.train(
    data='content/drive/MyDrive/dataset/data.yaml',  # Caminho para o arquivo YAML
    epochs=100,
    imgsz=640,
    batch=32,
    device='cpu',
    workers=2,
    name='sharped-vs-nonsharped-v1',
    pretrained=True,  # usa pesos pré-treinados
    augment=True,     # ativa augmentations básicas
    mosaic=1.0,       # intensidade do Mosaic
    mixup=0.2,        # intensidade do Mixup
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,  # variações de cor
    flipud=0.3, fliplr=0.5,  # flips verticais e horizontais
    degrees=10.0, translate=0.1, scale=0.5, shear=2.0,  # variações geométricas
    patience=20  # early stopping
)



## Inferências

### Carregar o modelo treinado

In [ ]:
from ultralytics import YOLO

# Carregar o modelo treinado
model = YOLO('/content/runs/detect/sharped-vs-nonsharped-v1/weights/best.pt')


###Fazer inferência em uma imagem

In [ ]:
results = model('/content/teste.jpg', conf=0.4)  # conf = limiar de confiança

# Exibir resultados
results[0].show()   # Visualização com bounding boxes
results[0].save('/content/teste_detectado.jpg')  # Salvar imagem com detecção


### 3. Fazer inferência em um vídeo

In [ ]:
results = model('/content/video.mp4', conf=0.4, save=True)

# Resultado será salvo na pasta /runs/detect/


4. Stream de webcam (ou outro dispositivo)

In [ ]:
model.predict(source=0, conf=0.4, show=True)  # source=0 = webcam padrão


### 5. Acessar objetos detectados programaticamente

In [ ]:
for r in results:
    boxes = r.boxes
    for box in boxes:
        cls = int(box.cls[0])   # ID da classe
        conf = float(box.conf[0])  # Confiança
        print(f'Classe: {cls}, Confiança: {conf:.2f}')
